In [71]:
import yfinance as yf
import pandas as pd
import numpy as np

import os

import datetime as dt
import utils
import matplotlib.pyplot as plt

import pandas_ta as ta

from skfolio import BasePortfolio, Population

In [2]:
DATA_PATH = '../../data/stocks'

In [100]:
a = 'CPSEETF.NS'
b = 'MON100.NS'

df = utils.download_and_cache([a, b], DATA_PATH)
df[a] = df[a].query('Volume > 0')
df[b] = df[b].query('Volume > 0')

#df[a].loc['2019-12-19':'2019-12-20', 'Close'] *= 10
df[b].loc['2021-06-17':'2021-06-18', 'Close'] *= 10

In [102]:
df_ret = pd.concat([df[x]
                    .Close
                    .pct_change()
                    .dropna()
                    .rename(x + '_ret') for x in [a, b]],
    axis=1).dropna()
df_ret

,CPSEETF.NS_ret,MON100.NS_ret
Date,,
2014-04-07,0.000517,-0.005467
2014-04-09,0.017045,-0.011041
2014-04-10,0.006602,0.003737
2014-04-11,-0.008073,-0.017591
2014-04-15,-0.003561,-0.006727
...,...,...
2024-08-19,0.009721,-0.004391
2024-08-20,-0.001264,0.015312
2024-08-21,0.000389,-0.002669


In [114]:
df_rsi = pd.concat([df[x][['Close']]
                    .ta.roc()
                    .dropna()
                    .rename(x + '_rsi') for x in [a, b]], axis=1).shift(1).dropna()
df_rsi

,CPSEETF.NS_rsi,MON100.NS_rsi
Date,,
2014-04-25,3.772607,-0.358362
2014-04-28,3.254128,0.706795
2014-04-29,0.914171,-1.616221
2014-04-30,-0.252267,-1.917352
2014-05-02,0.101732,-0.028425
...,...,...
2024-08-19,-4.078367,0.365185
2024-08-20,-1.607502,3.192668
2024-08-21,2.546176,11.093500


In [115]:
df_merged = pd.concat([df_ret, df_rsi], axis=1).dropna()
df_merged

,CPSEETF.NS_ret,MON100.NS_ret,CPSEETF.NS_rsi,MON100.NS_rsi
Date,,,,
2014-04-25,-0.004482,0.005164,3.772607,-0.358362
2014-04-28,-0.006003,-0.033853,3.254128,0.706795
2014-04-29,-0.005033,0.000665,0.914171,-1.616221
2014-04-30,-0.004552,0.001329,-0.252267,-1.917352
2014-05-02,0.008638,0.013741,0.101732,-0.028425
...,...,...,...,...
2024-08-19,0.009721,-0.004391,-4.078367,0.365185
2024-08-20,-0.001264,0.015312,-1.607502,3.192668
2024-08-21,0.000389,-0.002669,2.546176,11.093500


In [116]:
def func(row):
    if row[a + '_rsi'] > row[b + '_rsi']:
        return row[a + '_ret']
    else:
        return row[b + '_ret']

df_strat_ret = df_merged.apply(func, axis=1).to_frame()
df_strat_ret

,0
Date,
2014-04-25,-0.004482
2014-04-28,-0.006003
2014-04-29,-0.005033
2014-04-30,-0.004552
2014-05-02,0.008638
...,...
2024-08-19,-0.004391
2024-08-20,0.015312
2024-08-21,-0.002669


In [117]:

pop = Population([
    BasePortfolio(df_strat_ret.loc['2014':][0], df_strat_ret['2014':].index, name='Strategy'),
    BasePortfolio(df_ret['2014':][a+'_ret'], df_ret['2014':][a+'_ret'].index, name=a),
    BasePortfolio(df_ret['2014':][b+'_ret'], df_ret['2014':][b+'_ret'].index, name=b)
])

pop.set_portfolio_params(compounded=True)
pop.summary()

,Strategy,CPSEETF.NS,MON100.NS
Mean,0.12%,0.074%,0.088%
Annualized Mean,30.46%,18.72%,22.30%
Variance,0.020%,0.020%,0.021%
Annualized Variance,5.04%,4.92%,5.19%
Semi-Variance,0.010%,0.010%,0.011%
Annualized Semi-Variance,2.60%,2.58%,2.72%
Standard Deviation,1.41%,1.40%,1.44%
Annualized Standard Deviation,22.46%,22.18%,22.79%
Semi-Deviation,1.02%,1.01%,1.04%
Annualized Semi-Deviation,16.13%,16.07%,16.51%


In [118]:
pop.plot_cumulative_returns(log_scale=True)